In [ ]:
import joblib

# Load the saved model
best_mlp_loaded = joblib.load('best_mlp_model.pkl')


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MLPRegressor from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
import pandas as pd
df = pd.read_csv('tendons.csv')
df

,0,0.1,0.2
0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000
...,...,...,...
21444,2.123548,-0.558505,5.474580
21445,2.123548,2.436556,2.890726
21446,2.129685,5.370244,0.325283
21447,2.129685,8.015473,-2.184922


In [ ]:
# Predict the output using the trained MLP model
predictions = best_mlp_loaded.predict(features_scaled)



In [ ]:
negative_values = predictions < 0
negative_values.sum()

0

In [ ]:
# Add predictions as a new column in the DataFrame
df['predicted_current'] = predictions
predictions=np.clip(predictions, 0, 350)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Activation
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.layers import Dropout

# Define sliding window parameters
window_size = 20  # How many previous time steps to look at for each prediction
step_size = 1     # How much to slide the window at each step (can be 1 or more)
batch_size = 32   # Number of sequences processed in parallel during each training iteration

# Prepare the dataset with sliding window
def create_sliding_window_data(df, window_size, step_size):
    X = []
    y = []
    for i in range(0, len(df) - window_size, step_size):
        X.append(df[['0', '0.1', '0.2']].iloc[i:i+window_size].values)  # Tendon lengths for each window
        y.append(df['predicted_current'].iloc[i+window_size])  # Output current for the next time step
    return np.array(X), np.array(y)

# Generate sliding window dataset
X, y = create_sliding_window_data(df, window_size, step_size)

# Standardize the features
"""
scaler = StandardScaler()

X_reshaped = X.reshape(-1, X.shape[2])  # Flatten to 2D for standardization
X_scaled = scaler.fit_transform(X_reshaped)
X_scaled = X_scaled.reshape(X.shape)  # Reshape back to the original 3D shape"""

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the RNN Model
model = Sequential()
model.add(SimpleRNN(64, input_shape=(window_size, 3), activation='relu'))  # 64 hidden units, input shape matches (window_size, number of features)
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer to predict the current
model.add(Activation('relu'))  # Activation function for the output layer
# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size,
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
train_loss = model.evaluate(X_train, y_train)
test_loss = model.evaluate(X_test, y_test)

# Print results
print(f"Training Loss: {train_loss}")
print(f"Test Loss: {test_loss}")

# Predictions on the test set
y_pred = model.predict(X_test)

# You can now use `y_pred` to compare predicted currents with the actual currents


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Define the model
model = Sequential()

# Add the first RNN layer
model.add(SimpleRNN(64, activation='relu', input_shape=(window_size, 3), return_sequences=True))

# Add the second RNN layer (stacked)
model.add(SimpleRNN(64, activation='relu', return_sequences=True))

# Add the third RNN layer (stacked)
model.add(SimpleRNN(64, activation='relu'))

# Add the output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Summary of the model
model.summary()

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size,
                    validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
train_loss = model.evaluate(X_train, y_train)
test_loss = model.evaluate(X_test, y_test)

# Print results
print(f"Training Loss: {train_loss}")
print(f"Test Loss: {test_loss}")

# Predictions on the test set
y_pred = model.predict(X_test)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Predict on the test set (for RNN)
y_test_pred = model.predict(X_test)  # Use your RNN model

# Predict on the training set
y_train_pred = model.predict(X_train)  # Use your RNN model

# Evaluate the optimized model on the test set
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

# Evaluate the optimized model on the training set
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

# Print the results for training and test sets
print(f"Optimized RNN Training MSE: {train_mse}")
print(f"Optimized RNN Training MAE: {train_mae}")
print(f"Optimized RNN Training R² Score: {train_r2}")
print(f"Optimized RNN Test MSE: {test_mse}")
print(f"Optimized RNN Test MAE: {test_mae}")
print(f"Optimized RNN Test R² Score: {test_r2}")


134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
536/536 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
Optimized RNN Training MSE: 567.2464482994635
Optimized RNN Training MAE: 15.117171744737654
Optimized RNN Training R² Score: 0.990863665453288
Optimized RNN Test MSE: 568.4831160315683
Optimized RNN Test MAE: 15.101853514969214
Optimized RNN Test R² Score: 0.9909479310535034
